In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
# from test_utils import *
import test_utils
from utils import *

In [2]:
# Lx, Lt = 4, 8
# Lx, Lt = 2, 8
Lx, Lt = 4, 4
L = Lattice(Lx, Lt)
LL = np.array([Lx, Lx, Lx, Lt])
V = (Lx ** 3) * Lt

## NPR test

In [3]:
import xml.etree.ElementTree as ET
# npr_file = '/Users/theoares/lqcd/chroma/npr_tests/npr_out_wilson.dat.xml'
# npr_file = '/Users/theoares/lqcd/chroma/npr_tests/npr_out_clover.dat.xml'
# npr_file = '/Users/theoares/lqcd/chroma/npr_tests/npr_out_dwf.dat.xml'
npr_file = '/Users/theoares/lqcd/chroma/npr_tests/npr_0nubb.dat.xml'
tree = ET.parse(npr_file)
root = tree.getroot()
inline = root[5]
# npr = inline[3][0]
npr = inline[8][0]    # since NPR is running in zero_nubb_test.ini.xml
# npr = inline[10][0]
ft_prop = npr[8]  # this guy has the propagators
# prop_corr = npr[9]

In [4]:
def parse_prop_matrix(full_mat):
    prop_mat = np.zeros((Nd, Nd, Nc, Nc), dtype = np.complex64)
    for ii in range(Nd * Nd):
        dirac_index = full_mat[ii].attrib
        i, j = int(dirac_index['row']), int(dirac_index['col'])
        color_mat = full_mat[ii][0] # length 9
        for cc in range(Nc * Nc):
            color_index = color_mat[cc].attrib
            c, d = int(color_index['row']), int(color_index['col'])
            re_elem = color_mat[cc][0].text
            im_elem = color_mat[cc][1].text
            prop_mat[i, j, c, d] = np.complex(float(re_elem), float(im_elem))
    return prop_mat

In [5]:
mom_list = []
prop_list = []
for ii in range(0, len(ft_prop), 2):
    mom_ii_txt = ft_prop[ii][0].text
    mom_ii = np.array([int(x) for x in mom_ii_txt.split()])
    mom_list.append(mom_ii)
    prop_txt = ft_prop[ii + 1][0]
    prop_ii = parse_prop_matrix(prop_txt)
    prop_list.append(prop_ii)
mom_list = np.array(mom_list)
prop_list = np.array(prop_list)

In [6]:
epsilon = 1e-10
prop_list[np.abs(prop_list) < epsilon] = 0
for ii in range(len(mom_list)):
    print(mom_list[ii])
    if np.abs(np.sum(prop_list[ii])) < 1e-5:
        print('Prop == 0')
    else:
        nonzero_idx = ii
        print(ii)
        print(prop_list[ii])

[ 0  0 -2 -2]
Prop == 0
[-1 -1 -1 -2]
Prop == 0
[ 0 -1 -1 -2]
Prop == 0
[ 1 -1 -1 -2]
Prop == 0
[-1  0 -1 -2]
Prop == 0
[ 0  0 -1 -2]
Prop == 0
[ 1  0 -1 -2]
Prop == 0
[-1  1 -1 -2]
Prop == 0
[ 0  1 -1 -2]
Prop == 0
[ 1  1 -1 -2]
Prop == 0
[ 0 -2  0 -2]
Prop == 0
[-1 -1  0 -2]
Prop == 0
[ 0 -1  0 -2]
Prop == 0
[ 1 -1  0 -2]
Prop == 0
[-2  0  0 -2]
Prop == 0
[-1  0  0 -2]
Prop == 0
[ 0  0  0 -2]
Prop == 0
[ 1  0  0 -2]
Prop == 0
[ 2  0  0 -2]
Prop == 0
[-1  1  0 -2]
Prop == 0
[ 0  1  0 -2]
Prop == 0
[ 1  1  0 -2]
Prop == 0
[ 0  2  0 -2]
Prop == 0
[-1 -1  1 -2]
Prop == 0
[ 0 -1  1 -2]
Prop == 0
[ 1 -1  1 -2]
Prop == 0
[-1  0  1 -2]
Prop == 0
[ 0  0  1 -2]
Prop == 0
[ 1  0  1 -2]
Prop == 0
[-1  1  1 -2]
Prop == 0
[ 0  1  1 -2]
Prop == 0
[ 1  1  1 -2]
Prop == 0
[ 0  0  2 -2]
Prop == 0
[-1 -1 -2 -1]
Prop == 0
[ 0 -1 -2 -1]
Prop == 0
[ 1 -1 -2 -1]
Prop == 0
[-1  0 -2 -1]
Prop == 0
[ 0  0 -2 -1]
Prop == 0
[ 1  0 -2 -1]
Prop == 0
[-1  1 -2 -1]
Prop == 0
[ 0  1 -2 -1]
Prop == 0
[ 1  1 -2 -1]
Pr

In [7]:
nonzero_idx = 159
bvec = np.array([0., 0., 0., 0.5])
q = mom_list[nonzero_idx]
print(q)
q_lat = np.sin(L.to_linear_momentum(q))
prop_q = prop_list[nonzero_idx]
prop_q_b = np.expand_dims(np.einsum('ijab->aibj', prop_q), 0) / L.vol
props_q_inv = invert_props(prop_q_b, Nb = 1)
Zq = quark_renorm(props_q_inv, q_lat, Nb = 1)
print(Zq[0])

[-1  0  0  0]
(1-3.2368587e-18j)


In [8]:
props_q_inv.shape

(1, 3, 4, 3, 4)

In [9]:
np.einsum('zaiaj->ij', prop_q_b) / 3

array([[ 0.4785235 +1.4393195e-18j,  0.        +0.0000000e+00j,
         0.        +0.0000000e+00j,  0.35503358+7.8743365e-19j],
       [ 0.        +0.0000000e+00j,  0.4785235 +1.4393195e-18j,
         0.35503358+7.8743365e-19j,  0.        +0.0000000e+00j],
       [ 0.        +0.0000000e+00j, -0.35503358-7.8743365e-19j,
         0.4785235 +1.4393195e-18j,  0.        +0.0000000e+00j],
       [-0.35503358-7.8743365e-19j,  0.        +0.0000000e+00j,
         0.        +0.0000000e+00j,  0.4785235 +1.4393195e-18j]],
      dtype=complex64)

In [ ]:
np.einsum('zaiaj->ij', props_q_inv) / 3

## Free field test

In [36]:
# test_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
# test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_105926/free_field.h5'
f1 = h5py.File(test_file, 'r')
Sk1_1 = f1['prop_k1/q1100'][()]
Sk2_1 = f1['prop_k2/q1100'][()]
Sq_1 = np.einsum('ijab->aibj', f1['prop_q/q1100'][()])
Sq_1b = np.expand_dims(Sq_1, axis = 0)
Sq_1_inv = invert_props(Sq_1b, Nb = 1)
GV1 = np.zeros((4, 4, 4, 3, 3), dtype = np.complex64)
GA1 = np.zeros((4, 4, 4, 3, 3), dtype = np.complex64)
Gn1 = np.zeros((16, 4, 4, 4, 4, 3, 3, 3, 3), dtype = np.complex64)
for mu in range(4):
    GV1[mu] = f1['GV' + str(mu + 1) + '/q1100'][()]
    GA1[mu] = f1['GA' + str(mu + 1) + '/q1100'][()]
for n in range(16):
    Gn1[n] = f1['Gn' + str(n) + '/q1100'][()]

In [35]:
f1['Gn0/q1100'][()].shape

(4, 4, 4, 4, 3, 3, 3, 3)

In [37]:
# test_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
# test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
test_file2 = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_4444/free_field.h5'
f2 = h5py.File(test_file2, 'r')
Sk1_2 = f2['prop_k1/q1100'][()]
Sk2_2 = f2['prop_k2/q1100'][()]
Sq_2 = np.einsum('ijab->aibj', f2['prop_q/q1100'][()])
Sq_2b = np.expand_dims(Sq_2, axis = 0)
Sq_2_inv = invert_props(Sq_2b, Nb = 1)
GV2 = np.zeros((4, 4, 4, 3, 3), dtype = np.complex64)
GA2 = np.zeros((4, 4, 4, 3, 3), dtype = np.complex64)
Gn2 = np.zeros((16, 4, 4, 4, 4, 3, 3, 3, 3), dtype = np.complex64)
for mu in range(4):
    GV2[mu] = f2['GV' + str(mu + 1) + '/q1100'][()]
    GA2[mu] = f2['GA' + str(mu + 1) + '/q1100'][()]
for n in range(16):
    Gn2[n] = f2['Gn' + str(n) + '/q1100'][()]

In [47]:
# compare mom proj propagators for each one-- they're all the same so far
print(np.max(np.abs(Sk1_1 - Sk1_2)))
print(np.max(np.abs(Sk2_1 - Sk2_2)))
print(np.max(np.abs(Sq_1 - Sq_2)))
print(np.max(np.abs(GV1 - GV2)))
print(np.max(np.abs(GA1 - GA2)))
print(np.max(np.abs(Gn1 - Gn2)))
print(np.max(np.abs(np.einsum('nijklabcd->nlkjidcba', Gn1) - Gn2)))

2.6179345913313057e-09
2.617934535820155e-09
2.617934535820155e-09
5.3725133e-19
1.3010426e-17
0.052900262
1.13150904e-07


In [4]:
# chroma_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/cfgEXAMPLE.h5'
chroma_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/cfgEXAMPLE.h5'
# chroma_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/tests/hdf5/cfgEXAMPLE_bvec_1.h5'
f2 = h5py.File(chroma_file, 'r')
Sk1_chroma = f2['prop_k1/q2200'][()]
Sk2_chroma = f2['prop_k2/q2200'][()]
Sq_chroma = f2['prop_q/q2200'][()]

Sq_chroma = np.einsum('ijab->aibj', f2['prop_q/q2200'][()])
Sq_chroma_b = np.expand_dims(Sq_chroma, axis = 0)
Sq_chroma_inv = invert_props(Sq_chroma_b, Nb = 1)

KeyError: "Unable to open object (object 'q2200' doesn't exist)"

In [ ]:
Sq_chroma_inv

In [ ]:
# test Zq-- why is it so small for chroma output?
bvec = np.array([0.0, 0.0, 0.0, 0.5], dtype = np.float32)
k1 = np.array([-2, 0, 2, 0])
k2 = np.array([0, 2, 2, 0])
q = k2 - k1
q_lat = np.sin(L.to_linear_momentum(q + bvec))
print('clover')
Zq_clover = quark_renorm(Sq_clover_inv, q_lat, Nb = 1)
print('Zq = ' + str(Zq_clover))

print('chroma dwf')
Zq_chroma = quark_renorm(Sq_chroma_inv, q_lat, Nb = 1)
print('Zq = ' + str(Zq_chroma))

In [ ]:
Sq_red = np.einsum('zaiaj->ij', Sq_clover_inv) / 3

In [ ]:
Sq_red

In [ ]:
gamma[3]

In [ ]:
Zq_clover

In [ ]:
Zq_chroma

## Play around with free field output, written explicitly. Use this one to use correct free field output

In [4]:
Lx, Lt = 4, 4

In [5]:
# Load in the free field propagators
# job_num = 30558    # this is the one that works well and produces the correct RCs
job_num = 106440
k1 = np.array([-1, 0, 1, 0])
k2 = np.array([0, 1, 1, 0])
# k1 = np.array([-2, 0, 2, 0])
# k2 = np.array([0, 2, 2, 0])
q = k2 - k1
q_lat = np.sin(L.to_linear_momentum(q + bvec))
qlat_slash = slash(q_lat)
# test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_' + str(job_num) + '/free_field.h5'
test_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_props_' + str(job_num) + '/free_field_props.h5'
# test_file = '/Users/poare/Dropbox (MIT)/research/0nubb/tests/hdf5/free_field_qlua_clover.h5'
prop_k1, prop_k2, prop_q = test_utils.read_propagators(test_file, L, q)

In [ ]:
# compare job 106440 with job 30558

In [6]:
# Fourier transform and momentum project
FTk1 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTk2 = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTmq = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
FTm2q = np.zeros((Lx, Lx, Lx, Lt), dtype = np.complex64)
for x, y, z in itertools.product(range(Lx), repeat = 3):
    for t in range(Lt):
        pos = np.array([x, y, z, t])
        FTk1[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k1 + bvec))
        FTk2[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, k2 + bvec))
        FTq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, q + bvec))
        FTmq[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -q))
        FTm2q[x, y, z, t] = np.exp(2 * np.pi * 1j * np.dot(pos / LL, -2 * q))

In [7]:
Sk1 = np.einsum('xyzt,xyzt...->...', FTk1, prop_k1) / V
Sk2 = np.einsum('xyzt,xyzt...->...', FTk2, prop_k2) / V
Sq = np.einsum('xyzt,xyzt...->...', FTq, prop_q) / V
antiprop_k2 = np.einsum('ik,...akbl,lj->...aibj', gamma5, adjoint(prop_k2), gamma5)
Sk1_inv = np.linalg.tensorinv(Sk1)
Sk2_inv = np.linalg.tensorinv(Sk2)
Sq_inv = np.linalg.tensorinv(Sq)

## Verify that Zq, ZV, and ZA are all equal to 1 for the free field

In [8]:
# Verify Zq is 1
# Zq = (1j) * sum([q[mu] * np.einsum('ij,ajai', analysis.gamma[mu], Sq_inv) for mu in range(analysis.d)]) \
#     / (12 * analysis.square(q))
Zq = (1j) * sum([q[mu] * np.einsum('ij,ajai', gamma[mu], Sq_inv) for mu in range(4)]) \
    / (12 * square(q))
print(Zq)    # for free field, Zq must be 1.

(1.0000000085571377-1.3877787807814457e-16j)


In [9]:
# Get and amputate three point functions
# GV = np.array([np.einsum('xyzt,xyztaick,kl,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], prop_k1) for mu in range(4)]) / V
# GA = np.array([np.einsum('xyzt,xyztaick,km,ml,xyztclbj->aibj', FTmq, antiprop_k2, analysis.gamma[mu], analysis.gamma5, prop_k1) for mu in range(4)]) / V
GV = np.array([np.einsum('xyzt,xyztaick,kl,xyztclbj->aibj', FTmq, antiprop_k2, gamma[mu], prop_k1) for mu in range(4)]) / V
GA = np.array([np.einsum('xyzt,xyztaick,km,ml,xyztclbj->aibj', FTmq, antiprop_k2, gamma[mu], gamma5, prop_k1) for mu in range(4)]) / V
ΓV = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GV[mu], Sk1_inv) for mu in range(4)])
ΓA = np.array([np.einsum('aibj,bjck,ckdl->aidl', Sk2_inv, GA[mu], Sk1_inv) for mu in range(4)])
qΓV = np.einsum('m,maibj->aibj', q_lat, ΓV)
qΓA = np.einsum('m,maibj->aibj', q_lat, ΓA)

In [10]:
# Verify axial and vector R.C.s are 1
# ZV = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,ji->', qΓV, qlat_slash)
# ZA = 12 * Zq * analysis.square(q_lat) / np.einsum('aiaj,jk,ki->', qΓA, analysis.gamma5, qlat_slash)
ZV = 12 * Zq * square(q_lat) / np.einsum('aiaj,ji->', qΓV, qlat_slash)
ZA = 12 * Zq * square(q_lat) / np.einsum('aiaj,jk,ki->', qΓA, gamma5, qlat_slash)
print(ZV)
print(ZA)

(0.9999999914428666+7.472878310435776e-17j)
(0.9999999914428678+2.2036006131531826e-16j)


## Play around with the propagators and find some way to get the amputated 4 point function equal to its tree level value.

In [11]:
# Confirm that I'm getting the right combinations of gamma matrices
A = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contr = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓO = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    #A[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaList[n], prop_k1)
    A[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, gammaList[n], prop_k1)
    An_contr = np.einsum('...ckdl,...aibj->...aibjckdl', A[n], A[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', A[n], A[n])
    GO[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contr) / V    # TODO later try with no momentum
    ΓO[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GO[n])  #z is n

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [37]:
# Use reverse gamma matrices that Phiala uses
AR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
# An_contrR = np.zeros((16, 4, 4, 4, 4, 3, 4, 3, 4), dtype = np.complex64)
GOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
ΓOR = np.zeros((16, 3, 4, 3, 4, 3, 4, 3, 4), dtype = np.complex64)
for n in range(16):
    print(n)
    #AR[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, analysis.gammaListRev[n], prop_k1)
    AR[n] = np.einsum('...aick,kl,...clbj->...aibj', antiprop_k2, gammaListRev[n], prop_k1)
    An_contrR = np.einsum('...ckdl,...aibj->...aibjckdl', AR[n], AR[n]) - np.einsum('...ckbj,...aidl->...aibjckdl', AR[n], AR[n])
    GOR[n] = 2 * np.einsum('xyzt,xyzt...->...', FTm2q, An_contrR) / V    # TODO later try with no momentum
    ΓOR[n] = np.einsum('aiem,gobj,ckfn,hpdl,emgofnhp->aibjckdl', Sk2_inv, Sk1_inv, Sk2_inv, Sk1_inv, GOR[n])  #z is n

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [38]:
for m in range(16):
    if np.allclose(A[m], AR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(A[m], -AR[m]):
        print('n = ' + str(m) + ', -')

n = 0, +
n = 1, +
n = 2, +
n = 3, -
n = 4, +
n = 5, -
n = 6, -
n = 7, -
n = 8, +
n = 9, -
n = 10, -
n = 11, -
n = 12, -
n = 13, -
n = 14, -
n = 15, +


In [39]:
for m in range(16):
    if np.allclose(GO[m], GOR[m]):
        print('n = ' + str(m) + ', +')
    elif np.allclose(GO[m], -GOR[m]):
        print('n = ' + str(m) + ', -')
    else:
        print('n = ' + str(m) + ', no match')

n = 0, +
n = 1, +
n = 2, +
n = 3, +
n = 4, +
n = 5, +
n = 6, +
n = 7, +
n = 8, +
n = 9, +
n = 10, +
n = 11, +
n = 12, +
n = 13, +
n = 14, +
n = 15, +


In [12]:
SS = ΓO[0]
PP = ΓO[15]
VV = ΓO[1] + ΓO[2] + ΓO[4] + ΓO[8]
# AA = -ΓO[14] + ΓO[13] - ΓO[11] + ΓO[7]    # SOMETHING WRONG WITH AA
# AA = -(ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7])
AA = ΓO[14] + ΓO[13] + ΓO[11] + ΓO[7]
TT = (ΓO[3] + ΓO[5] + ΓO[6] + ΓO[9] + ΓO[10] + ΓO[12])

# SS = ΓOR[0]
# PP = ΓOR[15]
# VV = ΓOR[1] + ΓOR[2] + ΓOR[4] + ΓOR[8]
# AA = ΓOR[14] + ΓOR[13] + ΓOR[11] + ΓOR[7]
# TT = (ΓOR[3] + ΓOR[5] + ΓOR[6] + ΓOR[9] + ΓOR[10] + ΓOR[12])

eps = 1e-5
SS[np.abs(SS) < eps] = 0
PP[np.abs(PP) < eps] = 0
VV[np.abs(VV) < eps] = 0
AA[np.abs(AA) < eps] = 0
TT[np.abs(TT) < eps] = 0
# Gamma_D = np.einsum('aiajckcl->ijkl', Gamma)

In [ ]:
# [a, i, b, j, c, k, d, l]
# SS[:, 0, :, 0, :, 1, :, 1]

In [13]:
Gamma = [VV + AA, VV - AA, SS - PP, SS + PP, TT]
# P = analysis.projectors()
P = projectors()
Lambda = np.einsum('nbjaidlck,maibjckdl->mn', P, Gamma)
# F = analysis.getF()
F = getF(L)

In [14]:
print(F)

[[3072.+0.j    0.+0.j    0.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j 2304.+0.j -384.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j -384.+0.j  576.+0.j    0.+0.j    0.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j  480.+0.j  288.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j  288.+0.j 2016.+0.j]]


In [47]:
np.ones?

In [15]:
eps = 1e-3
print((Lambda + eps * np.ones((5, 5))).astype(int))    # should equal F-- 

[[3072    0    0    0    0]
 [   0 2304 -383    0    0]
 [   0 -383  576    0    0]
 [   0    0    0  480  288]
 [   0    0    0  288 2016]]


/var/folders/q2/tk_jvj_106gddw96ph0kg7r40000gn/T/ipykernel_61150/861694132.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  print((Lambda + eps * np.ones((5, 5))).astype(int))    # should equal F--


In [16]:
for i in range(5):
    print(np.einsum('nbjaidlck,aibjckdl->n', P, Gamma[i]))

[ 3.072001e+03-1.1394893e-12j  2.632141e-04+0.0000000e+00j
 -2.002716e-04+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j]
[ 2.975464e-04+9.7787338e-14j  2.304000e+03-4.5474735e-13j
 -3.840001e+02+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j]
[ 0.0000000e+00+0.000000e+00j -3.8400009e+02+0.000000e+00j
  5.7600024e+02+0.000000e+00j  0.0000000e+00-7.640903e-16j
 -1.1444092e-04-7.640903e-16j]
[ 0.0000000e+00+0.000000e+00j  2.4604797e-04+0.000000e+00j
 -8.8691711e-05+0.000000e+00j  4.8000034e+02-3.121692e-15j
  2.8800012e+02-3.121692e-15j]
[   0.     +0.0000000e+00j    0.     +0.0000000e+00j
    0.     +0.0000000e+00j  288.00006-1.1668785e-14j
 2015.9998 -1.1668785e-14j]


In [37]:
GV.shape

(4, 3, 4, 3, 4)

In [41]:
Sk1.shape

(3, 4, 3, 4)

In [43]:
# # save output to file in free_field_4444 so I can run it through my renorm script
# out_file_free = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/free_field_4444/free_field.h5'
# f4444 = h5py.File(out_file_free, 'w')
# f4444['moms/q1100/k1'] = k1
# f4444['moms/q1100/k2'] = k2
# f4444['prop_k1/q1100'] = np.einsum('aibj->ijab', Sk1)
# f4444['prop_k2/q1100'] = np.einsum('aibj->ijab', Sk2)
# f4444['prop_q/q1100'] = np.einsum('aibj->ijab', Sq)
# for mu in range(4):
#     f4444['GV' + str(mu + 1) + '/q1100'] = np.einsum('aibj->ijab', GV[mu])
#     f4444['GA' + str(mu + 1) + '/q1100'] = np.einsum('aibj->ijab', GA[mu])
# for n in range(16):
#     f4444['Gn' + str(n) + '/q1100'] = np.einsum('aibjckdl->ijklabcd', GO[n])
# f4444.close()

## Run through the packaged renormalization procedure to see if Python code has a bug

## Compare with QLUA output. There seems to be a bug somewhere, since the 3-point functions don't match

In [ ]:
# Saving mqFT2, A_Gamma, and the Green's function. Compare all of them to what this code outputs. 
# when everything here matches, I should be able to feed QLUA output into free_field.py and have it work
qlua_jobnum = 30661
qlua_out_file = '/Users/theoares/Dropbox (MIT)/research/0nubb/meas/'+'free_field_'+str(qlua_jobnum)+'/free_field.h5'
fq = h5py.File(qlua_out_file, 'r')
mqFT2 = fq['mqFT2'][()]
A_gamma = np.einsum('...ijab->...aibj', np.array([fq['Agamma' + str(n)][()] for n in range(16)]))
qlua_GO = np.einsum('nijklabcd->naibjckdl', np.array([fq['Gn' + str(n) + '/q1100'][()] for n in range(16)]))

In [ ]:
# Compare FT factors from QLUA with Python
# print(mqFT2)
# print(FTm2q)
print(np.allclose(mqFT2, FTm2q))

In [ ]:
# Compare A_gamma output from QLUA and Python for some n.
n = 3
print(A_gamma[n])
print(A[n])

In [ ]:
# Compare QLUA GO output with Python GO output
# print(qlua_GO[n])
# print(GO[n])    # python output for GO at gamma = 3
print(np.allclose(qlua_GO[n], GO[n]))
print(np.allclose(np.einsum('aibjckdl->dlckbjai', qlua_GO[n]), GO[n]))